Create compression version of GPT-2 nad BERT model and save the memory =)

## cola example

The Semantic Textual Similarity Benchmark (Cer et al., 2017) is a collection of sentence pairs drawn from news headlines, video and image captions, and natural language inference data. Each pair is human-annotated with a similarity score from 0 to 5.

In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [2]:
#!pip install Partial State

In [3]:
import transformers
print (transformers.__version__)

4.31.0


In [4]:
#from sparse_grad_spars import replace_bert_layers #replace_bert_layers(model, UV_dict):

In [4]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast, BertConfig, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn

from functools import lru_cache

import matplotlib.pyplot as plt
import torch.autograd
import torch.nn.functional as F
import torch
from torch import Tensor
from torch.nn.parameter import Parameter,   UninitializedParameter
from torch.nn import functional as F

def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)



def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3

def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)

@lru_cache
def get_I_matrix(b, r):
    return  torch.eye(b*r).to('cuda')

def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3

def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

class LinearFunctionSparseGrad(torch.autograd.Function):

        # Note that forward, setseup_context, and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, weight, bias, U, VT):
        input = input @ U.T ## Here change
        ctx.save_for_backward(input, weight, bias, U, VT) # space 2
        ctx.size = input.shape[0]
        return  input @ weight.T @ VT.T + bias # space 2  # HERE change
    


    @staticmethod
    def backward(ctx, grad_output):

        input, weight, bias, U, VT = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None
        # print (grad_output.shape, VT.T.shape, U.shape)
        grad_output = grad_output @ VT# !!!! HERE change
        if ctx.needs_input_grad[0]:
            grad_input = grad_output @ weight @ U
        if ctx.needs_input_grad[1]:
            grad_weight = torch.einsum('ijk,kjl->il', grad_output.T, input)#grad_output.T @input
            #grad_weight = VT.T @ grad_weight  # !!!! HERE change
            grad_weight = torch.where(torch.abs(grad_weight) >= 0.001, grad_weight, torch.tensor(0.0).to('cuda')).to_sparse()  ## возвращаем градиент в каком пространстве?? VERY IMPORTANT
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output
            
        return grad_input, grad_weight, grad_bias, None, None, None, None, None

class SparseGradLinear(torch.nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = None #Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = None# Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.treshold = 1e-3
        self.U = None
        self.VT = None
        self.is_sparse = False
                
        
    def from_linear(self, linear: nn.Linear, tuple_UV = tuple(), transpose=False):
        if transpose:
            self.weight = torch.nn.Parameter(linear.weight.data.T)
        else:
            self.weight = torch.nn.Parameter(linear.weight.data)
        self.bias = torch.nn.Parameter(linear.bias.data.clone()) if linear.bias is not None else None
        self.U = tuple_UV[0]
        self.VT = tuple_UV[1]
        
    def rewert_to_linear(self):
        self.weight = torch.nn.Parameter(self.VT@self.weight@self.U)
        self.bias = torch.nn.Parameter(linear.bias.data.clone()) if linear.bias is not None else None
        self.is_sparse = False
        
        
        
    def forward(self, x):
        
        return LinearFunctionSparseGrad.apply(x, self.weight, self.bias, self.U, self.VT)

            

def replace_bert_layers(model, UV_dict):
    if hasattr(model, "bert") and hasattr(model.bert, "encoder"):
        encoder = model.bert.encoder
    elif hasattr(model, "encoder"):
        encoder = model.encoder
    else:
        raise ValueError("Expected model to have attribute 'encoder' or 'bert.encoder'.")

    for i, layer in enumerate(model.bert.encoder.layer):
        token_dim, hidden_dim = layer.intermediate.dense.weight.shape
        #print ("dense")
        #print ("old shape", token_dim, hidden_dim)
        
        new_layer = SparseGradLinear(token_dim, hidden_dim)

        new_layer.from_linear(layer.intermediate.dense, UV_dict['interm'])

        model.bert.encoder.layer[i].intermediate.dense = new_layer
          
        #print ("new shape", layer.intermediate.dense.weight.shape)
        
        token_dim, hidden_dim = layer.output.dense.weight.shape
        #print ("output")
        #print ("old shape", token_dim, hidden_dim)
        
        new_layer = SparseGradLinear(token_dim, hidden_dim)

        new_layer.from_linear(layer.output.dense, UV_dict['output'])

        model.bert.encoder.layer[i].output.dense = new_layer
          
        #print ("new shape", layer.output.dense.weight.shape)
        #print ("\n\n")

    return model








In [5]:
def sparse_grad_linear(model, UV_dict):
    print ("create bert with sparse grads")
    model = replace_bert_layers(model, UV_dict)
    print ("created bert with sparse grads")
    return model

In [6]:
from datasets import load_dataset
import pandas as pd

dataset_cola = load_dataset('glue', 'cola')
dataset_cola.num_rows

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 8551, 'validation': 1043, 'test': 1063}

In [7]:
label_list = dataset_cola["train"].features["label"].names
num_labels = len(label_list)


In [8]:
task_num_labels = num_labels

from transformers import AutoConfig, BertConfig, AutoModelForSequenceClassification, AutoTokenizer

path_name = r"bert-base-uncased"

config = AutoConfig.from_pretrained(
    path_name,
    num_labels=num_labels,
)

model = AutoModelForSequenceClassification.from_pretrained(
    path_name,
    config=config,
)

tokenizer = AutoTokenizer.from_pretrained(path_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#model=model.to('cuda:2')

In [10]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys['cola']

In [11]:

def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        
        
        result = tokenizer.batch_encode_plus(*args, max_length=128, truncation=True, padding="max_length")

        result["label"] = examples["label"]
        return result

In [12]:
dataset_cola['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8551
})

In [13]:
tokenized_dataset = dataset_cola.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=False
        )

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [14]:
tokenized_dataset['validation']

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1043
})

In [15]:
from transformers import Trainer
import evaluate as ev
import numpy as np

from transformers import TrainingArguments, Trainer, EvalPrediction

2023-08-07 19:50:58.982936: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [16]:
metric = ev.load("glue", 'cola')

In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def compute_metrics(p: EvalPrediction):
        preds_ = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds_ = np.argmax(preds_, axis=1)
        
        result = metric.compute(predictions=preds_, references=p.label_ids)
        if True:
            result["combined_score"] = np.mean(list(result.values())).item()
            return result
        else:
            return {"accuracy": (preds_ == p.label_ids).astype(np.float32).mean().item()}

In [19]:
training_args1 = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=100,
    max_steps = 11,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    save_steps=1000,
    overwrite_output_dir=True,
    output_dir="./bert_stsb_128",
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    seed=297104,
    report_to='none',
    )

training_args2 = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=1,
    evaluation_strategy="steps",
    skip_memory_metrics = False,
    eval_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    save_steps=1000,
    overwrite_output_dir=True,
    output_dir="./bert_stsb_128",
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    seed=297104,
    report_to='none',
    )

In [20]:
from trainers_custom import TrainerBert1, TrainerBert2, TrainerDoubleOpt

In [21]:
trainer = TrainerBert1(
        model=model,
        args=training_args1,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        compute_metrics = compute_metrics,
)
trainer.make_grad_bank()

In [22]:
trainer.train_dataset

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8551
})

## BERT sparse

In [23]:

print ("len small dataset", len(trainer.train_dataset))
print ("lets collect gradients")
train_result = trainer.train()
print ("finishing collecting gradients")
#model = create_space_matrices(model)


len small dataset 8551
lets collect gradients


/opt/.pyenv/versions/3.8.10/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


finishing collecting gradients


In [24]:
from sparse_grad_spars import Tucker_Decomposition
import torch

UV_dict = {}

grads1 = torch.stack(trainer.grads1[:120])
u1, VT, U = Tucker_Decomposition(grads1)
UV_dict.update({"output":tuple((U, VT))})
grads2 = torch.stack(trainer.grads2[:120])
u1, VT, U = Tucker_Decomposition(grads2)
UV_dict.update({"interm":tuple((U, VT))})

In [26]:
UV_dict 

{'output': (tensor([[-0.0129,  0.0006,  0.0082,  ...,  0.0036, -0.0028,  0.0070],
          [-0.0150,  0.0035,  0.0011,  ..., -0.0064, -0.0076,  0.0132],
          [-0.0213, -0.0019,  0.0082,  ...,  0.0535,  0.0548,  0.0326],
          ...,
          [-0.0157, -0.0049, -0.0033,  ..., -0.0045,  0.0199,  0.0064],
          [-0.0160, -0.0002,  0.0062,  ..., -0.0012, -0.0091,  0.0026],
          [-0.0148,  0.0024,  0.0001,  ...,  0.0073,  0.0149, -0.0224]],
         device='cuda:0'),
  tensor([[-0.0342,  0.0044,  0.0708,  ..., -0.0640, -0.0047, -0.0304],
          [-0.0393,  0.0189, -0.0805,  ...,  0.0219,  0.0082, -0.0442],
          [ 0.0227,  0.0022,  0.0022,  ..., -0.0007, -0.0198, -0.0420],
          ...,
          [-0.0361,  0.0103,  0.0275,  ...,  0.0426, -0.0097, -0.0442],
          [-0.0162, -0.0278, -0.0014,  ..., -0.0090, -0.0074, -0.0457],
          [-0.0237, -0.0225,  0.0053,  ...,  0.0009,  0.0142, -0.0326]],
         device='cuda:0')),
 'interm': (tensor([[-0.0058, -0.0033, 

In [25]:
model = sparse_grad_linear(model, UV_dict)

create bert with sparse grads
created bert with sparse grads


In [26]:
del grads1, grads2, trainer
torch.cuda.empty_cache()

In [27]:
trainer = TrainerDoubleOpt(
        model=model,
        args=training_args2,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        compute_metrics = compute_metrics,
)


#trainer.create_scheduler(num_training_steps = trainer.max_steps)

trainer.make_grad_bank()

In [ ]:
train_result = trainer.train()

params ['bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.1.intermediate.dense.weight', 'bert.encoder.layer.1.output.dense.weight', 'bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.3.output.dense.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.output.dense.weight', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.5.output.dense.weight', 'bert.encoder.layer.6.intermediate.dense.weight', 'bert.encoder.layer.6.output.dense.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.7.output.dense.weight', 'bert.encoder.layer.8.intermediate.dense.weight', 'bert.encoder.layer.8.output.dense.weight', 'bert.encoder.layer.9.intermediate.dense.weight', 'bert.encoder.layer.9.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', '

Step,Training Loss,Validation Loss,Matthews Correlation,Combined Score
100,No log,0.619977,0.000000,0.000000
200,No log,0.628406,0.000000,0.000000
300,No log,0.628442,0.000000,0.000000


In [30]:
trainer.log_metrics("train", train_result.metrics)

***** train metrics *****
  before_init_mem_cpu        =     6096MB
  before_init_mem_gpu        =      716MB
  epoch                      =        1.0
  init_mem_cpu_alloc_delta   =        0MB
  init_mem_cpu_peaked_delta  =        0MB
  init_mem_gpu_alloc_delta   =        0MB
  init_mem_gpu_peaked_delta  =        0MB
  total_flos                 =   523836GF
  train_loss                 =     0.6169
  train_mem_cpu_alloc_delta  =        4MB
  train_mem_cpu_peaked_delta =        0MB
  train_mem_gpu_alloc_delta  =      836MB
  train_mem_gpu_peaked_delta =     1747MB
  train_runtime              = 0:01:53.80
  train_samples_per_second   =     75.134
  train_steps_per_second     =      4.701


In [30]:
from transformers import  TrainerCallback
import torch
import numpy as np
import time

class ProfCallback(TrainerCallback):
    def __init__(self, prof):
        self.prof = prof

    def on_step_end(self, args, state, control, **kwargs):
        self.prof.step()


In [31]:
profiler_result_path = 'file.json'
def train_on_epoch(generator, model, loss_function, optimizer):
    model.train()
    epoch_loss = []
    for batch_of_x, batch_of_y in generator:
        optimizer.zero_grad()
        with torch.autograd.profiler.profile(use_cuda=True, record_shapes=True, with_flops=True, profile_memory=True) as prof:
            output = model(batch_of_x)
            loss = loss_function(output, batch_of_y)
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item()) 
    
    prof.export_chrome_trace(profiler_result_path)

In [32]:
import torch
with torch.profiler.profile(activities=[torch.profiler.ProfilerActivity.CPU,
                                        torch.profiler.ProfilerActivity.CUDA], 
                            schedule=torch.profiler.schedule(skip_first=3, wait=1, warmup=1, active=2, repeat=2),
                            on_trace_ready=torch.profiler.tensorboard_trace_handler('hf-training-sparse'),
                            profile_memory=True,
                            with_stack=True,
                            record_shapes=True) as prof:
    
    trainer.add_callback(ProfCallback(prof=prof))
    train_result = trainer.train()
    prof.export_chrome_trace(profiler_result_path)

params ['bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.1.intermediate.dense.weight', 'bert.encoder.layer.1.output.dense.weight', 'bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.3.output.dense.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.output.dense.weight', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.5.output.dense.weight', 'bert.encoder.layer.6.intermediate.dense.weight', 'bert.encoder.layer.6.output.dense.weight', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.7.output.dense.weight', 'bert.encoder.layer.8.intermediate.dense.weight', 'bert.encoder.layer.8.output.dense.weight', 'bert.encoder.layer.9.intermediate.dense.weight', 'bert.encoder.layer.9.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', '

Step,Training Loss,Validation Loss,Matthews Correlation,Combined Score
100,No log,0.618294,0.000000,0.000000
200,No log,0.621816,0.000000,0.000000
300,No log,0.633324,0.000000,0.000000
400,No log,0.618174,0.000000,0.000000
500,0.617600,0.618146,0.000000,0.000000


RuntimeError: Trace is already saved

In [33]:
trainer.evaluate()

{'eval_loss': 0.618505597114563,
 'eval_matthews_correlation': 0.0,
 'eval_combined_score': 0.0,
 'eval_runtime': 3.1871,
 'eval_samples_per_second': 327.252,
 'eval_steps_per_second': 2.824,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 28672,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 20480,
 'eval_mem_gpu_peaked_delta': 654792192}

In [ ]:
import pickle
d = pickle.load( open( "/notebook/compression/exps/cola_pickles/bert_eval.pkl", "rb" ) )

In [ ]:
e = pickle.load( open( "/notebook/compression/exps/cola_pickles/bert_eval_sp.pkl", "rb" ) ) 

In [ ]:
len(eval_losses)

In [ ]:
eval_losses.append(0.4231)

In [ ]:
eval_losses

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.array(d), label = "bert full")
plt.plot(np.array(eval_losses), label = "sparse bert sharing")
plt.plot(np.array(e), label = "sparse bert")
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pickle
d = pickle.load( open( "/notebook/compression/exps/cola_pickles/bert.pkl", "rb" ) ) 

plt.plot(np.mean(np.array(d)[:500].reshape(-1, 50), axis=1), label = "bert full")
plt.plot(np.mean(np.array(trainer.losses)[:500].reshape(-1, 50), axis=1), label = "sparse bert")
plt.legend()

In [ ]:
/notebook/compression/exps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.mean(np.array(d)[:500].reshape(-1, 50), axis=1), label = "bert full")
plt.plot(np.mean(np.array(trainer.losses)[:500].reshape(-1, 50), axis=1), label = "sparse bert")
plt.legend()

In [ ]:
eval_losses = [elem['eval_loss'] for elem in trainer.state.log_history if 'eval_loss' in elem.keys()]
import pickle
with open('/notebook/compression/exps/cola_pickles/bert_eval_sp_001.pkl', 'wb') as f:
    pickle.dump(np.array(eval_losses), f)

In [ ]:
#train_result = trainer.train()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.mean(np.array(d)[:500].reshape(-1, 50), axis=1), label = "bert full")
plt.plot(np.mean(np.array(trainer.losses)[:500].reshape(-1, 50), axis=1), label = "sparse bert")
plt.legend()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.log_metrics("train", train_result.metrics)

In [ ]:
class ProfCallback(TrainerCallback):
    def __init__(self, prof):
        self.prof = prof

    def on_step_end(self, args, state, control, **kwargs):
        self.prof.step()
